In [1]:
# Start with setting up the speech to text with Whisper API

In [ ]:
!pip install git+https://github.com/openai/whisper.git 
# Install Whisper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-7gkikgit
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-7gkikgit
  Resolved https://github.com/openai/whisper.git to commit 3e1780fd37686666f568be9c99f5b5e3e4f2eb92
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 16.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230124-py3-none-any.whl size=1179407 sha256=233ad9418b621716e76c66dd3b74e4169f2a521922b919c74fbfd099f6b48ee6
  Stored in directory: /tmp/pip-ephem-wheel-cache-fpnvfyym/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built opena

In [16]:
%%bash
# Audio Imports
# libav
apt-get install pip
pip install simpleaudio
#apt-get install libav-tools libavcodec-extra

####    OR    #####

# ffmpeg
#apt-get install ffmpeg libavcodec-extra

Reading package lists...
Building dependency tree...
Reading state information...
python3-pip is already the newest version (20.0.2-5ubuntu1.8).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for simpleaudio: filename=simpleaudio-1.0.4-cp38-cp38-linux_x86_64.whl size=2065466 sha256=72f91cefe6253857905441a8e758d052666ed73e3a9f7f4ed6e7b357e49768ab
  Stored in directory: /root/.cache/pip/wheels/3c/46/0a/194e1db8ba343c98e075410186eba0b3bce3993062cdb0169d
Successfully built simpleaudio


In [22]:
# Record Audio in Colab with Pydub
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
!pip -q install pydub
from pydub import AudioSegment

# Install playback function
from pydub import AudioSegment
from pydub.playback import play

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

# Record saves audio file of type pydub.audio_segment
def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  return audio

In [23]:
# Record File
myAudioFile = record(3)
# Unfortunately AudioSegment is having trouble playing audio that came from BytesIO; we'll need to fix this later.

<IPython.core.display.Javascript object>

SimpleaudioError: ignored

In [ ]:
# Get whisper working


# Install Whisper
!pip install git+https://github.com/openai/whisper.git 

# Bring down test audio
!pip install yt-dlp

In [32]:
!yt-dlp https://www.youtube.com/watch?v=esMEmqhErE0 --format m4a -o "%(id)s.%(ext)s"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-c21advze
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-c21advze
  Resolved https://github.com/openai/whisper.git to commit 3e1780fd37686666f568be9c99f5b5e3e4f2eb92
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[youtube] Extracting URL: https://www.youtube.com/watch?v=esMEmqhErE0
[youtube] esMEmqhErE0: Downloading webpage
[youtube] esMEmqhErE0: Downloading android player API JSON
[youtube] esMEmqhErE0: Downloading player 7862ca1f
[info] esMEmqhErE0: Downloading 1 format(s): 140
[dashsegments] Total fragments: 1
[download] Destination: esMEmqhErE0.m4a
[download] 100% of   70.08KiB in 00:00:00 at 259.92KiB/s
[FixupM4a] Correcting container of "esMEmqhErE0.m4a"


In [34]:
!whisper "/content/esMEmqhErE0.m4a" --model small --language English

2023-03-04 19:52:10.106761: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 19:52:11.447896: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-04 19:52:11.448026: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-04 19:52:11.448049: W tensorfl